In [2]:
import cv2
import numpy as np

In [3]:
def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.adaptiveThreshold(
        blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    return thresh

In [4]:

def find_largest_contour(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    best_cnt = None

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 1000:
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                best_cnt = approx
                max_area = area
    return best_cnt

In [5]:
def reorder(points):
    points = points.reshape((4, 2))
    new_points = np.zeros((4, 2), dtype=np.float32)

    sum_pts = points.sum(1)
    diff_pts = np.diff(points, axis=1)

    new_points[0] = points[np.argmin(sum_pts)]  # top-left
    new_points[2] = points[np.argmax(sum_pts)]  # bottom-right
    new_points[1] = points[np.argmin(diff_pts)]  # top-right
    new_points[3] = points[np.argmax(diff_pts)]  # bottom-left

    return new_points

In [6]:
def warp_perspective(img, points):
    ordered = reorder(points)
    dst = np.array([[0,0],[450,0],[450,450],[0,450]], dtype=np.float32)
    matrix = cv2.getPerspectiveTransform(ordered, dst)
    warped = cv2.warpPerspective(img, matrix, (450, 450))
    return warped

In [7]:
def capture_and_extract():
    cap = cv2.VideoCapture(0)
    print("📸 Press SPACE to capture the Sudoku grid...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to access webcam.")
            break
        
        cv2.imshow("Sudoku Capture", frame)
        key = cv2.waitKey(1)

        if key == 32:  # SPACE key
            thresh = preprocess_image(frame)
            grid_contour = find_largest_contour(thresh)

            if grid_contour is not None:
                warped = warp_perspective(frame, grid_contour)
                cv2.imwrite("sudoku_extracted.png", warped)
                print("✅ Sudoku grid extracted and saved as 'sudoku_extracted.png'")
                cv2.imshow("Extracted Grid", warped)
                cv2.waitKey(0)
            else:
                print("⚠️ Couldn't detect a proper Sudoku grid.")
            break

        elif key == 27:  # ESC key
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    capture_and_extract()

📸 Press SPACE to capture the Sudoku grid...
⚠️ Couldn't detect a proper Sudoku grid.
